<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# 训练神经网络模型
我们将训练我们的图像分类器来检测两个类
' free '和' blocked '，我们将使用它们来避免碰撞,而在此之前我们就在Pytorch初体验之简单的手势识别这一章节中简单使用过流行的深度学习库*PyTorch*利用Res-net18进行分类,
想必应该对此有一些理解了!

In [4]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

# 上传并提取数据集
上次，我们上一次收集并打包成.zip的数据集，然后通过调用shell （命令行）命令来提取（解压缩）此数据集。如果你直接就在jetson nano上训练，可以跳过这一步。

In [2]:
!unzip -q dataset.zip

replace dataset/blocked/a76f5bac-892f-11ea-b49c-4a4c3afdeae9.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


解压之后,你应该见到一个名为dataset的文件夹出现在文件浏览器上。

# 创建数据集实例

现在我们使用torchvision.datasets 库中的ImageFolder数据集类。里面有个附加torchvision.transforms库用于转换数据，为训练模型做好准备。

In [5]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

# 将数据集拆分为训练集和测试集
接下来，我们将数据集拆分为 训练集 和 测试集。测试集将用于验证我们训练完的模型准确性。

In [6]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 10, 10])

# 创建数据加载器以批量加载数据

我们将创建两个DataLoader实例，它们为洗牌数据提供实用程序，生成批次图像，并与多个任务并行加载样本。

In [7]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

# 定义神经网络
现在，我们定义我们将要训练的神经网络。 torchvision 库提供了一系列我们可以使用的预训练模型。

在一个称为迁移学习的过程中，我们可以重新利用预先训练的模型（在数百万图像上进行训练），以获得可能少的数据，准确完成尽量多的任务。

在预训练模型的原始训练中学到的重要特征可重复用于新任务。 我们将使用alexnet模型。

In [8]:
model = models.alexnet(pretrained=True)

和我们当初Pytorch初体验之简单的手势识别这一章节中使用的res-net18类似,alexnet模型最初是针对具有1000个类标签的数据集进行训练的，但我们的数据集只有两个类标签！ 我们将把最好的层替换为最新的，未经训练的层只有两个输出。

In [9]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

最后，我们将模型通过我们前面课程跟大家介绍或的'CUDA'转移到GPU上执行

In [10]:
device = torch.device('cuda')
model = model.to(device)

# 训练神经网络
使用下面的代码，将开始训练我们的神经网络，在运行完每个世代后，保存表现最佳的模型。

>一个世代是所有数据运行一遍

In [11]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

0: 0.600000
1: 0.500000
2: 0.500000
3: 0.500000
4: 0.500000
5: 0.500000
6: 0.500000
7: 0.500000
8: 0.500000
9: 0.500000
10: 0.500000
11: 0.500000
12: 0.500000
13: 0.500000
14: 0.500000
15: 0.500000
16: 0.500000
17: 0.500000
18: 0.500000
19: 0.500000
20: 0.500000
21: 0.500000
22: 0.500000
23: 0.500000
24: 0.500000
25: 0.500000
26: 0.500000
27: 0.500000
28: 0.500000
29: 0.500000


我们在上面的单元格代码执行完成之后,我们就可以看到目录中生成的名为 best_model.th 的模型,在下一个例程中我们将使用这个模型进行主动避障.